<a href="https://colab.research.google.com/github/sho-kawano/profit-gain/blob/master/Training_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install ktrain on Google Colab
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 127kB/s 
     |████████████████████████████████| 983kB 54.8MB/s 
     |████████████████████████████████| 245kB 56.8MB/s 
     |████████████████████████████████| 1.0MB 53.8MB/s 
     |████████████████████████████████| 471kB 52.1MB/s 
     |████████████████████████████████| 3.0MB 34.2MB/s 
     |████████████████████████████████| 1.1MB 38.1MB/s 
     |████████████████████████████████| 890kB 56.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.21.4-cp36-none-any.whl size=25270024 sha256=811b7d14b3fedf5d98694a9789236e5fd34aa16bdeb5a6b83a33408990c1b3e8
  Stored in directory: /root/.cache/pip/wheels/e8/b6/c2/a730bb7727f9402827eb9cdd277a527a4a88acb2b0d22f06f0
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=11c0db809d6aa2b655bbe29473868aef6ff14535224f426f4fc4dfef951cb4f9
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [ ]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

In [ ]:
ktrain.__version__

'0.21.4'

# Multiclass Text Classification Using BERT and Keras
In this example, we will use ***ktrain*** ([a lightweight wrapper around Keras](https://github.com/amaiya/ktrain)) to build a model using the dataset employed in the **scikit-learn** tutorial: [Working with Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).  Unlike in the tutorial, we will sample 20 newsgroups to create a relatively small multiclass text classification dataset.  The objective is to accurately classify each document into one of these 20 newsgroup topic categories.  This will provide us an opportunity to see **BERT** in action on a relatively smaller training set.  Let's fetch the [20newsgroups dataset ](http://qwone.com/~jason/20Newsgroups/) using scikit-learn.


In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'comp.sys.ibm.pc.hardware', 'rec.autos',
              'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey',
              'sci.med', 'sci.space', 'talk.politics.guns',
              'talk.politics.mideast']

train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 5756
size of validation set: 3831
classes: ['alt.atheism', 'comp.sys.ibm.pc.hardware', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.med', 'sci.space', 'talk.politics.guns', 'talk.politics.mideast']


## STEP 1:  Load and Preprocess the Data
Preprocess the data using the `texts_from_array function` (since the data resides in an array).
If your documents are stored in folders or a CSV file you can use the `texts_from_folder` or `texts_from_csv` functions, respectively.

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=train_b.target_names,
                                                                       preprocess_mode='bert',
                                                                       ngram_range=1,
                                                                       maxlen=350,
                                                                       random_state= 3,
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


## STEP 2:  Load the BERT Model and Instantiate a Learner object

In [ ]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


## STEP 3: Train the Model

We train using one of the three learning rates recommended in the BERT paper: *5e-5*, *3e-5*, or *2e-5*.
Alternatively, the ktrain Learning Rate Finder can be used to find a good learning rate by invoking `learner.lr_find()` and `learner.lr_plot()`, prior to training.
The `learner.fit_onecycle` method employs a [1cycle learning rate policy](https://arxiv.org/pdf/1803.09820.pdf).



In [ ]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
960/960 [==============================] - 490s 510ms/step - loss: 0.8229 - accuracy: 0.7561
Epoch 2/4
960/960 [==============================] - 489s 510ms/step - loss: 0.1397 - accuracy: 0.9600
Epoch 3/4
960/960 [==============================] - 489s 509ms/step - loss: 0.0393 - accuracy: 0.9897
Epoch 4/4
960/960 [==============================] - 489s 510ms/step - loss: 0.0112 - accuracy: 0.9977


We can use the `learner.validate` method to test our model against the validation set.
As we can see, BERT achieves a **95%** accuracy.

In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=train_b.target_names)

                          precision    recall  f1-score   support

             alt.atheism       0.90      0.93      0.92       319
comp.sys.ibm.pc.hardware       0.97      0.99      0.98       392
               rec.autos       0.92      0.96      0.94       396
         rec.motorcycles       0.93      0.90      0.91       398
      rec.sport.baseball       0.96      0.97      0.97       397
        rec.sport.hockey       0.99      0.96      0.98       399
                 sci.med       0.97      0.95      0.96       396
               sci.space       0.96      0.96      0.96       394
      talk.politics.guns       0.93      0.96      0.94       364
   talk.politics.mideast       0.99      0.93      0.96       376

                accuracy                           0.95      3831
               macro avg       0.95      0.95      0.95      3831
            weighted avg       0.95      0.95      0.95      3831



array([[298,   0,   1,   3,   1,   0,   5,   9,   2,   0],
       [  0, 387,   1,   1,   0,   0,   0,   1,   1,   1],
       [  0,   3, 381,   8,   0,   0,   1,   2,   1,   0],
       [  2,   3,  24, 359,   0,   0,   1,   0,   9,   0],
       [  2,   0,   1,   0, 385,   5,   1,   0,   3,   0],
       [  0,   0,   3,   1,   8, 385,   0,   0,   2,   0],
       [  3,   3,   0,   7,   0,   0, 378,   0,   3,   2],
       [  5,   4,   1,   2,   1,   0,   1, 377,   3,   0],
       [  4,   0,   1,   5,   2,   0,   1,   1, 350,   0],
       [ 18,   1,   0,   2,   2,   0,   0,   1,   4, 348]])

## How to Use Our Trained BERT Model

We can call the `learner.get_predictor` method to obtain a Predictor object capable of making predictions on new raw data.

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

['alt.atheism',
 'comp.sys.ibm.pc.hardware',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.med',
 'sci.space',
 'talk.politics.guns',
 'talk.politics.mideast']

In [ ]:
predictor.predict(test_b.data[0:1])

['rec.autos']

In [ ]:
# we can visually verify that our prediction of 'sci.med' for this document is correct
print(test_b.data[0])

From: oaddab@stdvax (DIRK BROER)
Subject: Re: Questions about insurance companies (esp. Geico)
News-Software: VAX/VMS VNEWS 1.4-b1  
Organization: Goddard Space Flight Center - Robotics Lab
Distribution: usa
Lines: 32

In article <66758@mimsy.umd.edu>, davew@cs.umd.edu (David G. Wonnacott) writes...
>I'm considering switching to Geico insurance, but have heard that
>they do not assign a specific agent for each policy or claim.  I was
>worried that this might be a real pain when you make a claim.  I have
>also heard that they try to get rid of you if you have an accident.
> 
>I'm interestend in determining whether or not these things are true.
>Has anyone out there with Geico made a claim?  I'd be interested in
>hearing whether or not you were satisfied with the service and whether
>you then had trouble renewing your policy.
> 
>I'm also interested in any good or bad stories about Liberty Mutual or
>State Farm.
> 
Geico supports (reads gives to police) Laser Radars.  They have been know

In [ ]:
# we predicted the correct label
print(test_b.target_names[test_b.target[0]])

rec.autos


The `predictor.save` and `ktrain.load_predictor` methods can be used to save the Predictor object to disk and reload it at a later time to make predictions on new data.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# let's save the predictor for later use
predictor.save('/content/gdrive/My Drive/Colab Notebooks/ML_Profit-Gain/10NewsGroups/BERT_predictor')

In [ ]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor('/content/gdrive/My Drive/Colab Notebooks/ML_Profit-Gain/10NewsGroups/BERT_predictor')

In [ ]:
# make a prediction on the same document to verify it still works
reloaded_predictor.predict(test_b.data[0:1])

['rec.autos']